In [2]:
# dr_run_normwear_sanity.py
# ------------------------------------------------------------
# NormWear 로드 드라이런 (0~3단계 통합)
# - 0-1: 체크포인트 경로/파일 sanity check
# - 0-2: 의존성(import/버전) 확인
# - 0-3: 모델 초기화 + 더미 입력 임베딩 추출(Shape 검증)
# ------------------------------------------------------------

import os
import sys
import json
import math
import traceback

# ======== [사용자 환경에 맞게 수정] ========
NORMWEAR_REPO = r"C:\Users\user\code\NormWear"  # 깃 클론한 NormWear 레포 루트
WEIGHTS_DIR   = r"C:\Users\user\code\SDPhysiology\weights\normwear"  # .pth 저장 폴더
MAIN_CKPT     = "normwear_last_checkpoint-15470-correct.pth"  # 인코더용(필수)
ZS_CKPT       = "normwear_msitf_zeroshot_last_checkpoint-5.pth"  # 제로샷(선택)
FORCE_CPU     = True   # 초기 드라이런은 True 권장(성공 후 False로 GPU 전환)
# ==========================================

def _print_header(msg):
    print("\n" + "="*80)
    print(msg)
    print("="*80)

def _check_file_exists(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"파일이 없습니다: {path}")
    size = os.path.getsize(path)
    if size == 0:
        raise OSError(f"파일 크기가 0바이트입니다(손상): {path}")
    print(f" - OK: {path} (size: {size/1024/1024:.2f} MB)")

def _append_sys_path_for_import(normwear_repo):
    # from NormWear.main_model import NormWearModel 가 되려면
    # sys.path에 NormWear의 "부모 폴더"가 있어야 합니다.
    parent = os.path.dirname(normwear_repo)
    if parent not in sys.path:
        sys.path.insert(0, parent)
    print(f" - sys.path에 추가됨: {parent}")

def _select_device(force_cpu=True):
    import torch
    if force_cpu:
        print(" - 장치: CPU (FORCE_CPU=True)")
        return torch.device("cpu")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f" - 장치: {device}")
    return device

def _sanity_imports():
    import torch, numpy  # noqa: F401
    print(" - import torch / numpy OK")
    print(f"   torch.version: {torch.__version__}")
    try:
        print(f"   torch.cuda.is_available: {torch.cuda.is_available()}")
    except Exception:
        pass

def _test_encoder_forward(weight_path, device, sampling_rate=120, bn=2, nvar=3, seconds=2):
    """
    더미 입력: [bn, nvar, T]  (T = sampling_rate * seconds)
    출력: [bn, nvar, P, 768]  형태 확인
    """
    import torch
    from NormWear.main_model import NormWearModel

    print("\n[모델 초기화]")
    model = NormWearModel(weight_path=weight_path, optimized_cwt=True).to(device)
    model.eval()
    print(" - 모델 초기화 OK")

    T = sampling_rate * seconds
    x = torch.rand(bn, nvar, T, dtype=torch.float32, device=device)
    print(f"[더미 입력] x.shape={tuple(x.shape)}  (bn={bn}, nvar={nvar}, T={T}, sr={sampling_rate})")

    with torch.no_grad():
        out = model.get_embedding(x, sampling_rate=sampling_rate, device=device)  # [bn, nvar, P, 768]
    print(f"[임베딩 출력] out.shape={tuple(out.shape)}  (예상: [bn, nvar, P, 768])")

    # 간단한 형태 검증
    assert out.ndim == 4, f"출력 차원이 4가 아닙니다: {out.ndim}"
    assert out.shape[0] == bn, f"bn 불일치: {out.shape[0]} vs {bn}"
    assert out.shape[1] == nvar, f"nvar 불일치: {out.shape[1]} vs {nvar}"
    assert out.shape[-1] == 768, f"임베딩 차원(마지막)이 768이 아닙니다: {out.shape[-1]}"

    # 예시: 패치 평균 + 채널 평균 풀링 (나중에 회귀용 2D 임베딩)
    emb_patch_mean = out.mean(dim=2)         # [bn, nvar, 768]
    emb_channel_mean = emb_patch_mean.mean(dim=1)  # [bn, 768]
    print(f"[풀링 예시] channel-mean 임베딩 shape: {tuple(emb_channel_mean.shape)}")

    print("✅ 0-3단계 드라이런(인코더) 통과")

def main():
    try:
        _print_header("0-1) 체크포인트 경로/파일 sanity check")
        main_ckpt_path = os.path.join(WEIGHTS_DIR, MAIN_CKPT)
        _check_file_exists(main_ckpt_path)
        # 제로샷은 선택 사항이라 존재만 확인(사용은 안 함)
        zs_ckpt_path = os.path.join(WEIGHTS_DIR, ZS_CKPT)
        if os.path.exists(zs_ckpt_path):
            print(" - (선택) 제로샷 ckpt도 발견됨")
            _check_file_exists(zs_ckpt_path)
        else:
            print(" - (선택) 제로샷 ckpt는 현재 없음(문제 아님)")

        _print_header("0-2) 의존성 확인(import/버전)")
        _sanity_imports()

        _print_header("0-3) 모델 초기화 및 더미 임베딩 추출")
        _append_sys_path_for_import(NORMWEAR_REPO)
        device = _select_device(force_cpu=FORCE_CPU)

        # 더미 테스트: sr=120, bn=2, nvar=3(PPG/EDA/RSP 가정), 2초 길이
        _test_encoder_forward(
            weight_path=main_ckpt_path,
            device=device,
            sampling_rate=120,
            bn=2,
            nvar=3,
            seconds=2
        )

    except Exception as e:
        print("\n❌ 드라이런 실패:")
        print(" - 에러 메시지:", str(e))
        print(" - traceback ↓")
        traceback.print_exc()
        print("\n[빠른 점검 가이드]")
        print(" 1) NORMWEAR_REPO 경로가 올바른지(레포 루트) 확인")
        print(" 2) WEIGHTS_DIR/MODEL_CKPT 경로 및 파일명 오탈자 확인")
        print(" 3) from NormWear.main_model import NormWearModel 임포트가 가능한지")
        print(" 4) FORCE_CPU=True로 CPU에서 먼저 시도(GPU/CUDA 충돌 회피)")
        print(" 5) 더미 입력 shape가 [bn, nvar, T]인지, sampling_rate 인자를 맞게 넣었는지")
        sys.exit(1)

if __name__ == "__main__":
    main()



0-1) 체크포인트 경로/파일 sanity check
 - OK: C:\Users\user\code\SDPhysiology\weights\normwear\normwear_last_checkpoint-15470-correct.pth (size: 1552.57 MB)
 - (선택) 제로샷 ckpt도 발견됨
 - OK: C:\Users\user\code\SDPhysiology\weights\normwear\normwear_msitf_zeroshot_last_checkpoint-5.pth (size: 220.10 MB)

0-2) 의존성 확인(import/버전)
 - import torch / numpy OK
   torch.version: 2.5.1
   torch.cuda.is_available: True

0-3) 모델 초기화 및 더미 임베딩 추출
 - sys.path에 추가됨: C:\Users\user\code
 - 장치: CPU (FORCE_CPU=True)


c:\Users\user\anaconda3\envs\normwear\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\anaconda3\envs\normwear\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)



[모델 초기화]


C:\Users\user\code\NormWear\main_model.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stat_dict = torch.load(weight_path, map_location=torch.device('cpu'))['model']


Model Checkpoint is successfully loaded!
 - 모델 초기화 OK
[더미 입력] x.shape=(2, 3, 240)  (bn=2, nvar=3, T=240, sr=120)
[임베딩 출력] out.shape=(2, 3, 339, 768)  (예상: [bn, nvar, P, 768])
[풀링 예시] channel-mean 임베딩 shape: (2, 768)
✅ 0-3단계 드라이런(인코더) 통과


In [2]:
DATA_DIR   = r"D:\Labroom\SDPhysiology\Data\processed_individual_anonymized"
PATTERN    = "_Main.pkl"              # 파일명 패턴
SUBJECTS   = list(range(1, 109))      # 1~108
FS_RAW     = 120                      # 현재 샘플링 (이미 동기화됨)
FS_TARGET  = 50                       # 통일 샘플링 (권장)
WIN_SEC    = 30                       # 윈도우 길이
STRIDE_SEC = 2                        # 윈도우 간격(= hop)
LAG_MAX_S  = 3                        # 라벨 라그 탐색 범위
COLS       = dict(                    # 컬럼명 매핑(실제 df 컬럼에 맞게 수정 가능)
    PPG="PPG", RSP="RSP", EDA="EDA", PUPIL="Pupil", LABEL="anxiety"
)
CHANNEL_ORDER = ["PPG","RSP","EDA","PUPIL"]  # 고정 순서


In [3]:
import os, pandas as pd, numpy as np

def load_subject(pid:int)->pd.DataFrame:
    f = os.path.join(DATA_DIR, f"{pid:03d}{PATTERN}")
    df = pd.read_pickle(f)
    # 필수 컬럼 체크
    need = [COLS[c] for c in ["PPG","RSP","EDA","PUPIL","LABEL"]]
    missing = [c for c in need if c not in df.columns]
    assert not missing, f"PID {pid}: missing cols {missing}"
    # 기본 정리
    df = df.copy()
    df["pid"] = pid
    # NaN 처리(임시): 라벨 NaN 드롭, 신호 NaN은 선형보간(짧은 결측만)
    df = df[df[COLS["LABEL"]].notna()]
    df[need] = df[need].interpolate(limit=FS_RAW//2, limit_direction="both")
    return df

# 샘플 2~3개만 빠르게 검사
for pid in [1,2,3]:
    df = load_subject(pid)
    print(pid, df.shape, "NaN ratio:", df[ [COLS[k] for k in ["PPG","RSP","EDA","PUPIL","LABEL"]] ].isna().mean().round(4).to_dict())


AssertionError: PID 1: missing cols ['PPG', 'RSP', 'EDA', 'Pupil']

In [4]:
df = pd.read_pickle(r"D:\Labroom\SDPhysiology\Data\processed_individual_anonymized\001_Main.pkl")

In [6]:
df.columns

Index(['Frame', 'validL', 'validR', 'gazeoriginL_X', 'gazeoriginL_Y',
       'gazeoriginL_Z', 'gazeoriginR_X', 'gazeoriginR_Y', 'gazeoriginR_Z',
       'gazeL_X', 'gazeL_Y', 'gazeL_Z', 'gazeR_X', 'gazeR_Y', 'gazeR_Z',
       'pupilL', 'pupilR', 'eye_opennessL', 'eye_opennessR',
       'pupilLSensorPosL_X', 'pupilLSensorPosL_Y', 'pupilLSensorPosL_Z',
       'pupilLSensorPosR_X', 'pupilLSensorPosR_Y', 'pupilLSensorPosR_Z',
       'unit', 'unit_1', 'unit_2', 'unit_3', 'unit_4', 'unit_5', 'unit_6',
       'unit_7', 'unit_8', 'unit_9', 'unit_10', 'unit_11', 'unit_12',
       'unit_13', 'unit_14', 'unit_15', 'unit_16', 'unit_17', 'unit_18',
       'unit_19', 'unit_20', 'unit_21', 'unit_22', 'unit_23', 'unit_24',
       'unit_25', 'unit_26', 'unit_27', 'unit_28', 'unit_29', 'unit_30',
       'unit_31', 'unit_32', 'unit_33', 'unit_34', 'unit_35', 'unit_36',
       'X_pos', 'Y_pos', 'Z_pos', 'X_rot', 'Y_rot', 'Z_rot', 'PPG_Raw',
       'PPG_Clean', 'PPG_Rate', 'PPG_Quality', 'PPG_Peaks', 'EDA_R